In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from jpasLAEs.load_jpas_catalogs import load_minijpas_jnep
from load_paus_cat import load_paus_cat
from paus_utils import w_central
from jpasLAEs.utils import flux_to_mag
from jpasLAEs.utils import w_central as w_central_jpas

from astropy.coordinates import SkyCoord
import astropy.units as u

import numpy as np

In [ ]:
# Load miniJPAS
mj_cat_dir = '/home/alberto/cosmos/LAEs/csv'
mj_cat = load_minijpas_jnep(mj_cat_dir)
mj_cat['r_mag'] = flux_to_mag(mj_cat['pm_flx'][-2], w_central_jpas[-2])

# Load PAUS
field_name = 'W3'
path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}.csv']
paus_cat = load_paus_cat(path_to_cat)

mask_NB_number = (paus_cat['NB_number'] > 39)
paus_cat['flx'] = paus_cat['flx'][:, mask_NB_number]
paus_cat['err'] = paus_cat['err'][:, mask_NB_number]
paus_cat['NB_mask'] = paus_cat['NB_mask'][:, mask_NB_number]
for key in paus_cat.keys():
    if key in ['flx', 'err', 'NB_mask', 'area']:
        continue
    paus_cat[key] = paus_cat[key][mask_NB_number]

stack_nb_ids = np.arange(12, 26 + 1)
synth_BB_flx = np.average(paus_cat['flx'][stack_nb_ids],
                          weights=paus_cat['err'][stack_nb_ids] ** -2,
                          axis=0)

paus_cat['synth_r_mag'] = flux_to_mag(synth_BB_flx, w_central[-4])
paus_cat['r_mag'] = flux_to_mag(paus_cat['flx'][-4], w_central[-4])

In [ ]:
# Xmatch
coords_paus = SkyCoord(ra=paus_cat['RA'] * u.deg,
                       dec=paus_cat['DEC'] * u.deg)
coords_mj = SkyCoord(ra=mj_cat['RA'] * u.deg,
                       dec=mj_cat['DEC'] * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_mj)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1 * u.arcsec)

N_xm = sum(mask_dist)
print(f'N_miniJPAS = {len(mj_cat["r_mag"])}')
print(f'N_PAUS_W3 = {len(paus_cat["ref_id"])}')
print(f'Matched = {N_xm}')

In [ ]:
mj_interp_flx = np.empty((40, N_xm))
xm_id_masked = xm_id[mask_dist]

for src in range(N_xm):
    mj_interp_flx[:, src] = np.interp(w_central[:40], w_central_jpas[1:56],
                                      mj_cat['pm_flx'][1:56, xm_id_masked[src]])

In [ ]:
def ergscmAA_to_Jy(paus_flx_mat):
    c = 299792458 * u.m / u.s
    paus_flx_mat_Jy = np.empty_like(paus_flx_mat)
    flx_units = u.erg / (u.cm**2 * u.AA * u.s)
    for i in range(40):
        wavelength = w_central[i] * u.AA
        paus_flx_mat_Jy[i] = (paus_flx_mat[i] * flx_units / c * wavelength**2).to(u.Jy)

    return paus_flx_mat_Jy

In [ ]:
f1 = ergscmAA_to_Jy(paus_cat['flx'][:40, mask_dist])
f2 = ergscmAA_to_Jy(mj_interp_flx)

amp = np.average(f1 * f2, axis=0) / np.average(f1 * f1, axis=0)

fig, ax = plt.subplots(figsize=(8, 6))

bins = np.linspace(-0.5, 2, 50)

ax.hist(amp, bins=bins, histtype='step', label='All')
print(np.median(amp))

mask = (paus_cat['sg_flag'] == 1)[mask_dist]
ax.hist(amp[mask], bins=bins, histtype='step', label='Stars')

mask = (paus_cat['sg_flag'] == 0)[mask_dist] & (mj_cat['r_mag'][xm_id][mask_dist] > 22)
ax.hist(amp[mask], bins=bins, histtype='step', label='Gal. $r>22$')

mask = (paus_cat['sg_flag'] == 0)[mask_dist] & (mj_cat['r_mag'][xm_id][mask_dist] < 22)
ax.hist(amp[mask], bins=bins, histtype='step', label='Gal. $r<22$')

ax.legend()

ax.set_xlabel('Amp. miniJPAS/PAUS')

plt.show()

In [ ]:
f1 = ergscmAA_to_Jy(paus_cat['flx'][:40, mask_dist])[:20]
f2 = ergscmAA_to_Jy(mj_interp_flx)[:20]

amp = np.average(f1 * f2, axis=0) / np.average(f1 * f1, axis=0)

fig, ax = plt.subplots(figsize=(8, 6))

bins = np.linspace(-0.5, 2, 50)

ax.hist(amp, bins=bins, histtype='step', label='All')
print(np.median(amp))

mask = (paus_cat['sg_flag'] == 1)[mask_dist]
ax.hist(amp[mask], bins=bins, histtype='step', label='Stars')

mask = (paus_cat['sg_flag'] == 0)[mask_dist] & (mj_cat['r_mag'][xm_id][mask_dist] > 22)
ax.hist(amp[mask], bins=bins, histtype='step', label='Gal. $r>22$')

mask = (paus_cat['sg_flag'] == 0)[mask_dist] & (mj_cat['r_mag'][xm_id][mask_dist] < 22)
ax.hist(amp[mask], bins=bins, histtype='step', label='Gal. $r<22$')

ax.legend()

ax.set_xlabel('Amp. miniJPAS/PAUS')
ax.set_title('20 Bluest filters')

plt.show()

In [ ]:
f1 = ergscmAA_to_Jy(paus_cat['flx'][:40, mask_dist])[20:]
f2 = ergscmAA_to_Jy(mj_interp_flx)[20:]

amp = np.average(f1 * f2, axis=0) / np.average(f1 * f1, axis=0)

fig, ax = plt.subplots(figsize=(8, 6))

bins = np.linspace(-0.5, 2, 50)

ax.hist(amp, bins=bins, histtype='step', label='All')
print(np.median(amp))

mask = (paus_cat['sg_flag'] == 1)[mask_dist]
ax.hist(amp[mask], bins=bins, histtype='step', label='Stars')

mask = (paus_cat['sg_flag'] == 0)[mask_dist] & (mj_cat['r_mag'][xm_id][mask_dist] > 22)
ax.hist(amp[mask], bins=bins, histtype='step', label='Gal. $r>22$')

mask = (paus_cat['sg_flag'] == 0)[mask_dist] & (mj_cat['r_mag'][xm_id][mask_dist] < 22)
ax.hist(amp[mask], bins=bins, histtype='step', label='Gal. $r<22$')

ax.legend()

ax.set_xlabel('Amp. miniJPAS/PAUS')
ax.set_title('20 Reddest filters')

plt.show()

In [ ]:
src_i = np.random.randint(0, N_xm)
src = np.where(mask_dist)[0][src_i]

fig, ax = plt.subplots()

ax.errorbar(w_central[:40], paus_cat['flx'][:40, src],
            yerr=paus_cat['err'][:40, src], label='PAUS')
ax.errorbar(w_central_jpas[:56], mj_cat['pm_flx'][:56, xm_id[src]],
            yerr=mj_cat['pm_err'][:56, xm_id[src]], label='miniJPAS')
ax.plot(w_central[:40], mj_interp_flx[:, src_i], label='miniJPAS interpolated')

ax.legend(fontsize=10)

ax.set_ylabel('$f_\lambda$ [erg\,/\,(s\,cm$^{2}$\,\AA)]')
ax.set_xlabel('Wavelength [\AA]')

plt.show()